In [1]:
import json
import csv
import tweepy
import re
import nltk
from nltk.stem import WordNetLemmatizer
import pandas as pd
import GetOldTweets3 as got
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
import numpy as np

In [2]:
df=pd.read_csv('../../Datasets/Twitter/Datasets with likes/final_srm_withlikes.csv')
df

,Unnamed: 0,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score
0,0,Oh snappic.twitter.com/xxMMCPGGFJ,ZAM the Kitesooner,1228465788773904384,2020-02-14 23:47:22,[],Srm_the_Kitsune,0,1,NaN,0.0000,Neutral,3
1,1,Great job Santa Rosa Branch! TGIF!!\n.\n.\n.\n...,SMRestorationService,1228443676411662336,2020-02-14 22:19:30,"['werestore', 'servicemasterrestore', 'SRM', '...",SVMRestoration,1,0,NaN,0.8753,Positive,5
2,2,I saw that on MSNBC or CNN very early this s m...,June Rzosa,1228437748274663424,2020-02-14 21:55:57,[],RzosaJune,0,0,NaN,-0.3818,Negative,2
3,3,Set an example-you first! Start re-distributi...,S.R. Mauricio,1228427424171511808,2020-02-14 21:14:55,[],columbus_srm,0,0,NaN,0.5411,Positive,4
4,4,Another great cover done by yours truly... #Re...,Giuseppe D.,1228421427293872128,2020-02-14 20:51:05,"['Repost', 'newmusic', 'newmusicfriday', 'danc...",giuseppedmusic,0,1,NaN,0.7906,Positive,5
5,5,*double ear infection- confirmed with my 2nd v...,sara,1228403195552182272,2020-02-14 19:38:39,[],ssssara_xo,0,0,NaN,0.0000,Neutral,3
6,6,Available worldwide now and new album coming s...,Ray Guell,1228398941609263104,2020-02-14 19:21:44,"['newmusic', 'newmusicfriday', 'dancemusic', '...",rayguell,0,2,NaN,0.0000,Neutral,3
7,7,The Best PUA Books On Meeting Womenhttp://rviv...,Bryce @ SelfReliantMan.com,1228394851055669248,2020-02-14 19:05:29,[],brycefoster_srm,0,0,NaN,0.6369,Positive,5
8,8,No More Ferrying By Private Car: Gandhi Family...,Muhundaram_SeethaRaman,1228384074013630464,2020-02-14 18:22:40,[],SRM_MUHUNDARAM,0,0,NaN,0.0516,Positive,3
9,9,Great results for our 1st team netballers and ...,St Edmund’s Sport,1228377686495682560,2020-02-14 17:57:17,"['beallyoucanbesport', 'rest', 'recovery']",StEdmundsSport,2,5,NaN,0.8313,Positive,5


In [3]:
df=df.drop(['Unnamed: 0'],axis=1)

In [4]:
def normalize(feature):
    max_value = df[feature].max()
    min_value = df[feature].min()
    result = (df[feature] - min_value) / (max_value - min_value)
    return result

In [5]:
df['retweets']=df['retweets'].astype(float)
df['likes']=df['likes'].astype(float)

In [6]:
df['retweets_n']=normalize('retweets')
df['likes_n']=normalize('likes')
df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n
0,Oh snappic.twitter.com/xxMMCPGGFJ,ZAM the Kitesooner,1228465788773904384,2020-02-14 23:47:22,[],Srm_the_Kitsune,0.0,1.0,NaN,0.0000,Neutral,3,0.000000,0.000138
1,Great job Santa Rosa Branch! TGIF!!\n.\n.\n.\n...,SMRestorationService,1228443676411662336,2020-02-14 22:19:30,"['werestore', 'servicemasterrestore', 'SRM', '...",SVMRestoration,1.0,0.0,NaN,0.8753,Positive,5,0.000713,0.000000
2,I saw that on MSNBC or CNN very early this s m...,June Rzosa,1228437748274663424,2020-02-14 21:55:57,[],RzosaJune,0.0,0.0,NaN,-0.3818,Negative,2,0.000000,0.000000
3,Set an example-you first! Start re-distributi...,S.R. Mauricio,1228427424171511808,2020-02-14 21:14:55,[],columbus_srm,0.0,0.0,NaN,0.5411,Positive,4,0.000000,0.000000
4,Another great cover done by yours truly... #Re...,Giuseppe D.,1228421427293872128,2020-02-14 20:51:05,"['Repost', 'newmusic', 'newmusicfriday', 'danc...",giuseppedmusic,0.0,1.0,NaN,0.7906,Positive,5,0.000000,0.000138
5,*double ear infection- confirmed with my 2nd v...,sara,1228403195552182272,2020-02-14 19:38:39,[],ssssara_xo,0.0,0.0,NaN,0.0000,Neutral,3,0.000000,0.000000
6,Available worldwide now and new album coming s...,Ray Guell,1228398941609263104,2020-02-14 19:21:44,"['newmusic', 'newmusicfriday', 'dancemusic', '...",rayguell,0.0,2.0,NaN,0.0000,Neutral,3,0.000000,0.000275
7,The Best PUA Books On Meeting Womenhttp://rviv...,Bryce @ SelfReliantMan.com,1228394851055669248,2020-02-14 19:05:29,[],brycefoster_srm,0.0,0.0,NaN,0.6369,Positive,5,0.000000,0.000000
8,No More Ferrying By Private Car: Gandhi Family...,Muhundaram_SeethaRaman,1228384074013630464,2020-02-14 18:22:40,[],SRM_MUHUNDARAM,0.0,0.0,NaN,0.0516,Positive,3,0.000000,0.000000
9,Great results for our 1st team netballers and ...,St Edmund’s Sport,1228377686495682560,2020-02-14 17:57:17,"['beallyoucanbesport', 'rest', 'recovery']",StEdmundsSport,2.0,5.0,NaN,0.8313,Positive,5,0.001426,0.000688


In [7]:
df['engagement_score']=60 * df['retweets_n'] + 40 * df['likes_n']

In [8]:
def clean_tweet(tweet): 
        ''' 
        Utility function to clean tweet text by removing links, special characters 
        using simple regex statements. 
        '''
        return ' '.join(re.sub("([^0-9A-Za-z \t]) |(\w+:\/\/\S+)", " ", tweet).split()) #(@[A-Za-z0-9]+)|

In [9]:
df['clean_tweets']=df['text'].map(clean_tweet)
df['text']=df['text'].map(str.lower)

In [10]:
from nltk.probability import FreqDist
from nltk import pos_tag, word_tokenize
from nltk.stem.porter import PorterStemmer
import matplotlib.pyplot as plt

from sklearn.manifold import MDS
from sklearn.metrics import euclidean_distances

In [11]:
df["clean_tweets"] = df["clean_tweets"].astype(str)
df["tweet_tokens"] = df["clean_tweets"].apply(lambda each_post: word_tokenize(re.sub(r'[^\w\s]',' ',each_post.lower())))
df["tweet_tokens"] = df["tweet_tokens"].apply(lambda list_of_words: [x for x in list_of_words if x not in stopwords])
df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens
0,oh snappic.twitter.com/xxmmcpggfj,ZAM the Kitesooner,1228465788773904384,2020-02-14 23:47:22,[],Srm_the_Kitsune,0.0,1.0,NaN,0.0000,Neutral,3,0.000000,0.000138,0.005501,Oh snappic.twitter.com/xxMMCPGGFJ,"[oh, snappic, twitter, com, xxmmcpggfj]"
1,great job santa rosa branch! tgif!!\n.\n.\n.\n...,SMRestorationService,1228443676411662336,2020-02-14 22:19:30,"['werestore', 'servicemasterrestore', 'SRM', '...",SVMRestoration,1.0,0.0,NaN,0.8753,Positive,5,0.000713,0.000000,0.042766,Great job Santa Rosa Branch TGIF!! . . . . . ....,"[great, job, santa, rosa, branch, tgif, werest..."
2,i saw that on msnbc or cnn very early this s m...,June Rzosa,1228437748274663424,2020-02-14 21:55:57,[],RzosaJune,0.0,0.0,NaN,-0.3818,Negative,2,0.000000,0.000000,0.000000,I saw that on MSNBC or CNN very early this s m...,"[saw, msnbc, cnn, early, 2, group, kids, stood..."
3,set an example-you first! start re-distributi...,S.R. Mauricio,1228427424171511808,2020-02-14 21:14:55,[],columbus_srm,0.0,0.0,NaN,0.5411,Positive,4,0.000000,0.000000,0.000000,Set an example-you first Start re-distributing...,"[set, example, first, start, distributing, wea..."
4,another great cover done by yours truly... #re...,Giuseppe D.,1228421427293872128,2020-02-14 20:51:05,"['Repost', 'newmusic', 'newmusicfriday', 'danc...",giuseppedmusic,0.0,1.0,NaN,0.7906,Positive,5,0.000000,0.000138,0.005501,Another great cover done by yours truly.. #Rep...,"[another, great, cover, done, truly, repost, r..."
5,*double ear infection- confirmed with my 2nd v...,sara,1228403195552182272,2020-02-14 19:38:39,[],ssssara_xo,0.0,0.0,NaN,0.0000,Neutral,3,0.000000,0.000000,0.000000,*double ear infection confirmed with my 2nd vi...,"[double, ear, infection, confirmed, 2nd, visit..."
6,available worldwide now and new album coming s...,Ray Guell,1228398941609263104,2020-02-14 19:21:44,"['newmusic', 'newmusicfriday', 'dancemusic', '...",rayguell,0.0,2.0,NaN,0.0000,Neutral,3,0.000000,0.000275,0.011001,Available worldwide now and new album coming s...,"[available, worldwide, new, album, coming, soo..."
7,the best pua books on meeting womenhttp://rviv...,Bryce @ SelfReliantMan.com,1228394851055669248,2020-02-14 19:05:29,[],brycefoster_srm,0.0,0.0,NaN,0.6369,Positive,5,0.000000,0.000000,0.000000,The Best PUA Books On Meeting,"[best, pua, books, meeting]"
8,no more ferrying by private car: gandhi family...,Muhundaram_SeethaRaman,1228384074013630464,2020-02-14 18:22:40,[],SRM_MUHUNDARAM,0.0,0.0,NaN,0.0516,Positive,3,0.000000,0.000000,0.000000,No More Ferrying By Private Car Gandhi Family ...,"[ferrying, private, car, gandhi, family, set, ..."
9,great results for our 1st team netballers and ...,St Edmund’s Sport,1228377686495682560,2020-02-14 17:57:17,"['beallyoucanbesport', 'rest', 'recovery']",StEdmundsSport,2.0,5.0,NaN,0.8313,Positive,5,0.001426,0.000688,0.113034,Great results for our 1st team netballers and ...,"[great, results, 1st, team, netballers, boys, ..."


In [12]:
nltk.download('averaged_perceptron_tagger')
df['pos']=df['tweet_tokens'].apply(nltk.tag.pos_tag)
df

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/Manam/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens,pos
0,oh snappic.twitter.com/xxmmcpggfj,ZAM the Kitesooner,1228465788773904384,2020-02-14 23:47:22,[],Srm_the_Kitsune,0.0,1.0,NaN,0.0000,Neutral,3,0.000000,0.000138,0.005501,Oh snappic.twitter.com/xxMMCPGGFJ,"[oh, snappic, twitter, com, xxmmcpggfj]","[(oh, UH), (snappic, NN), (twitter, NN), (com,..."
1,great job santa rosa branch! tgif!!\n.\n.\n.\n...,SMRestorationService,1228443676411662336,2020-02-14 22:19:30,"['werestore', 'servicemasterrestore', 'SRM', '...",SVMRestoration,1.0,0.0,NaN,0.8753,Positive,5,0.000713,0.000000,0.042766,Great job Santa Rosa Branch TGIF!! . . . . . ....,"[great, job, santa, rosa, branch, tgif, werest...","[(great, JJ), (job, NN), (santa, NN), (rosa, N..."
2,i saw that on msnbc or cnn very early this s m...,June Rzosa,1228437748274663424,2020-02-14 21:55:57,[],RzosaJune,0.0,0.0,NaN,-0.3818,Negative,2,0.000000,0.000000,0.000000,I saw that on MSNBC or CNN very early this s m...,"[saw, msnbc, cnn, early, 2, group, kids, stood...","[(saw, NN), (msnbc, NN), (cnn, VBD), (early, J..."
3,set an example-you first! start re-distributi...,S.R. Mauricio,1228427424171511808,2020-02-14 21:14:55,[],columbus_srm,0.0,0.0,NaN,0.5411,Positive,4,0.000000,0.000000,0.000000,Set an example-you first Start re-distributing...,"[set, example, first, start, distributing, wea...","[(set, NN), (example, NN), (first, JJ), (start..."
4,another great cover done by yours truly... #re...,Giuseppe D.,1228421427293872128,2020-02-14 20:51:05,"['Repost', 'newmusic', 'newmusicfriday', 'danc...",giuseppedmusic,0.0,1.0,NaN,0.7906,Positive,5,0.000000,0.000138,0.005501,Another great cover done by yours truly.. #Rep...,"[another, great, cover, done, truly, repost, r...","[(another, DT), (great, JJ), (cover, NN), (don..."
5,*double ear infection- confirmed with my 2nd v...,sara,1228403195552182272,2020-02-14 19:38:39,[],ssssara_xo,0.0,0.0,NaN,0.0000,Neutral,3,0.000000,0.000000,0.000000,*double ear infection confirmed with my 2nd vi...,"[double, ear, infection, confirmed, 2nd, visit...","[(double, JJ), (ear, JJ), (infection, NN), (co..."
6,available worldwide now and new album coming s...,Ray Guell,1228398941609263104,2020-02-14 19:21:44,"['newmusic', 'newmusicfriday', 'dancemusic', '...",rayguell,0.0,2.0,NaN,0.0000,Neutral,3,0.000000,0.000275,0.011001,Available worldwide now and new album coming s...,"[available, worldwide, new, album, coming, soo...","[(available, JJ), (worldwide, JJ), (new, JJ), ..."
7,the best pua books on meeting womenhttp://rviv...,Bryce @ SelfReliantMan.com,1228394851055669248,2020-02-14 19:05:29,[],brycefoster_srm,0.0,0.0,NaN,0.6369,Positive,5,0.000000,0.000000,0.000000,The Best PUA Books On Meeting,"[best, pua, books, meeting]","[(best, JJS), (pua, NN), (books, NNS), (meetin..."
8,no more ferrying by private car: gandhi family...,Muhundaram_SeethaRaman,1228384074013630464,2020-02-14 18:22:40,[],SRM_MUHUNDARAM,0.0,0.0,NaN,0.0516,Positive,3,0.000000,0.000000,0.000000,No More Ferrying By Private Car Gandhi Family ...,"[ferrying, private, car, gandhi, family, set, ...","[(ferrying, VBG), (private, JJ), (car, NN), (g..."
9,great results for our 1st team netballers and ...,St Edmund’s Sport,1228377686495682560,2020-02-14 17:57:17,"['beallyoucanbesport', 'rest', 'recovery']",StEdmundsSport,2.0,5.0,NaN,0.8313,Positive,5,0.001426,0.000688,0.113034,Great results for our 1st team netballers and ...,"[great, results, 1st, team, netballers, boys, ...","[(great, JJ), (results, NNS), (1st, CD), (team..."


In [13]:
# Extract the list of nouns
df['nouns']=df['pos'].apply(lambda pos_list: [x[0] for x in pos_list if x[1].startswith('NN')])
df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens,pos,nouns
0,oh snappic.twitter.com/xxmmcpggfj,ZAM the Kitesooner,1228465788773904384,2020-02-14 23:47:22,[],Srm_the_Kitsune,0.0,1.0,NaN,0.0000,Neutral,3,0.000000,0.000138,0.005501,Oh snappic.twitter.com/xxMMCPGGFJ,"[oh, snappic, twitter, com, xxmmcpggfj]","[(oh, UH), (snappic, NN), (twitter, NN), (com,...","[snappic, twitter, com, xxmmcpggfj]"
1,great job santa rosa branch! tgif!!\n.\n.\n.\n...,SMRestorationService,1228443676411662336,2020-02-14 22:19:30,"['werestore', 'servicemasterrestore', 'SRM', '...",SVMRestoration,1.0,0.0,NaN,0.8753,Positive,5,0.000713,0.000000,0.042766,Great job Santa Rosa Branch TGIF!! . . . . . ....,"[great, job, santa, rosa, branch, tgif, werest...","[(great, JJ), (job, NN), (santa, NN), (rosa, N...","[job, santa, rosa, branch, tgif, disasterresto..."
2,i saw that on msnbc or cnn very early this s m...,June Rzosa,1228437748274663424,2020-02-14 21:55:57,[],RzosaJune,0.0,0.0,NaN,-0.3818,Negative,2,0.000000,0.000000,0.000000,I saw that on MSNBC or CNN very early this s m...,"[saw, msnbc, cnn, early, 2, group, kids, stood...","[(saw, NN), (msnbc, NN), (cnn, VBD), (early, J...","[saw, msnbc, group, kids, arm, praise, trump, ..."
3,set an example-you first! start re-distributi...,S.R. Mauricio,1228427424171511808,2020-02-14 21:14:55,[],columbus_srm,0.0,0.0,NaN,0.5411,Positive,4,0.000000,0.000000,0.000000,Set an example-you first Start re-distributing...,"[set, example, first, start, distributing, wea...","[(set, NN), (example, NN), (first, JJ), (start...","[set, example, start, wealth, billionaire, hyp..."
4,another great cover done by yours truly... #re...,Giuseppe D.,1228421427293872128,2020-02-14 20:51:05,"['Repost', 'newmusic', 'newmusicfriday', 'danc...",giuseppedmusic,0.0,1.0,NaN,0.7906,Positive,5,0.000000,0.000138,0.005501,Another great cover done by yours truly.. #Rep...,"[another, great, cover, done, truly, repost, r...","[(another, DT), (great, JJ), (cover, NN), (don...","[cover, rayguell, album, edm, musicismylife, n..."
5,*double ear infection- confirmed with my 2nd v...,sara,1228403195552182272,2020-02-14 19:38:39,[],ssssara_xo,0.0,0.0,NaN,0.0000,Neutral,3,0.000000,0.000000,0.000000,*double ear infection confirmed with my 2nd vi...,"[double, ear, infection, confirmed, 2nd, visit...","[(double, JJ), (ear, JJ), (infection, NN), (co...","[infection, visit, clinic, week]"
6,available worldwide now and new album coming s...,Ray Guell,1228398941609263104,2020-02-14 19:21:44,"['newmusic', 'newmusicfriday', 'dancemusic', '...",rayguell,0.0,2.0,NaN,0.0000,Neutral,3,0.000000,0.000275,0.011001,Available worldwide now and new album coming s...,"[available, worldwide, new, album, coming, soo...","[(available, JJ), (worldwide, JJ), (new, JJ), ...","[album, edm, srm, miami, beach, florida]"
7,the best pua books on meeting womenhttp://rviv...,Bryce @ SelfReliantMan.com,1228394851055669248,2020-02-14 19:05:29,[],brycefoster_srm,0.0,0.0,NaN,0.6369,Positive,5,0.000000,0.000000,0.000000,The Best PUA Books On Meeting,"[best, pua, books, meeting]","[(best, JJS), (pua, NN), (books, NNS), (meetin...","[pua, books, meeting]"
8,no more ferrying by private car: gandhi family...,Muhundaram_SeethaRaman,1228384074013630464,2020-02-14 18:22:40,[],SRM_MUHUNDARAM,0.0,0.0,NaN,0.0516,Positive,3,0.000000,0.000000,0.000000,No More Ferrying By Private Car Gandhi Family ...,"[ferrying, private, car, gandhi, family, set, ...","[(ferrying, VBG), (private, JJ), (car, NN), (g...","[car, gandhi, family, queue, security, swarajy..."
9,great results for our 1st team netballers and ...,St Edmund’s Sport,1228377686495682560,2020-02-14 17:57:17,"['beallyoucanbesport', 'rest', 'recovery']",StEdmundsSport,2.0,5.0,NaN,0.8313,Positive,5,0.001426,0.000688,0.113034,Great results for our 1st team netballers and ...,"[great, results, 1st, team, netballers, boys, ...","[(great, JJ), (r

In [14]:
#Filter the noun list only to make the list of attributes 
noun_lst = df['nouns'].apply(pd.Series).stack().unique()

# # get the frequencies for the nouns:
noun_df= pd.DataFrame(noun_lst)
noun_df.columns=['nouns']
noun_df

,nouns
0,snappic
1,twitter
2,com
3,xxmmcpggfj
4,job
5,santa
6,rosa
7,branch
8,tgif
9,disasterrestoration


In [15]:
## get frequency of all words
all_words = df['nouns'].sum()
freq_dist = nltk.FreqDist(all_words)

df_fdist=pd.DataFrame(list(freq_dist.items()), columns=['term', 'freq'])

noun_freq_df = pd.merge(noun_df, df_fdist, right_on='term', left_on='nouns')
noun_freq_df

,nouns,term,freq
0,snappic,snappic,2
1,twitter,twitter,3644
2,com,com,3623
3,xxmmcpggfj,xxmmcpggfj,2
4,job,job,195
5,santa,santa,13
6,rosa,rosa,13
7,branch,branch,55
8,tgif,tgif,2
9,disasterrestoration,disasterrestoration,35


In [16]:
noun_freq_df=noun_freq_df.sort_values(by=['freq'], ascending=False)
noun_freq_df

,nouns,term,freq
48,srm,srm,6734
1,twitter,twitter,3644
2,com,com,3623
73,pic,pic,1524
187,university,university,1308
26,time,time,682
263,day,day,577
136,students,students,550
368,people,people,467
237,technology,technology,440


In [17]:
noun_freq_df.to_csv('nouns_tweets.csv')

In [18]:
#list of irreleveant tweet topics
l=['vairamuthu','ban','suicides','indievideogames','movies','bjp','justice','hindu','death']
df['irrelevance']=df['nouns'].apply(lambda x: len(set(x).intersection(l)))
df[df.irrelevance!=0]

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens,pos,nouns,irrelevance
139,@irctcofficial yhi no.... hai available irctc ...,Ishant,1227786781778759680,2020-02-13 02:49:14,[],Ish6octgmailco1,0.0,0.0,NaN,0.0000,Neutral,3,0.000000,0.000000,0.000000,@IRCTCofficial yhi no... Hai available irctc o...,"[irctcofficial, yhi, hai, available, irctc, oc...","[(irctcofficial, JJ), (yhi, NN), (hai, NN), (a...","[yhi, hai, pe, bhi, srm, kro, bjp, pic, twitte...",1
267,ab srm ho to ..\nparliament m ek bill le aao o...,Anil Saini,1227225463396950016,2020-02-11 13:38:45,[],AnilSai34140355,0.0,0.0,NaN,0.0000,Neutral,3,0.000000,0.000000,0.000000,Ab srm ho to .. Parliament m ek bill le aao or...,"[ab, srm, ho, parliament, ek, bill, le, aao, a...","[(ab, NN), (srm, NN), (ho, NN), (parliament, N...","[ab, srm, ho, parliament, ek, bill, act, bna, ...",1
464,"pttv is dmk’s mouthpiece, blatantly ly they su...",Ramesh Kasturi Rangan,1226380905465278464,2020-02-09 05:42:47,[],RameshKasturiR1,1.0,1.0,NaN,0.4019,Positive,4,0.000713,0.000138,0.048266,PTTV is DMK’s mouthpiece blatantly ly they sup...,"[pttv, dmk, mouthpiece, blatantly, ly, support...","[(pttv, NN), (dmk, NN), (mouthpiece, NN), (bla...","[pttv, dmk, mouthpiece, support, dmk, discussi...",1
509,srm is 1st 80 to 100cr\nban is 1st 110 to 120c...,the-DHFM-ever,1226139547769028608,2020-02-08 13:43:43,[],UrstrulyShiva09,2.0,2.0,NaN,-0.3034,Negative,2,0.001426,0.000275,0.096532,SRM Is 1st 80 To 100Cr BAN Is 1st 110 To 120Cr...,"[srm, 1st, 80, 100cr, ban, 1st, 110, 120cr, sl...","[(srm, NN), (1st, CD), (80, CD), (100cr, CD), ...","[srm, ban, sln, wise, syera, rs, consider, che...",1
651,massive: nirbhaya's rapists to be hanged on ja...,SARATH,1214529222045290496,2020-01-07 12:48:26,[],sarath_srm,0.0,0.0,NaN,-0.8481,Negative,1,0.000000,0.000000,0.000000,MASSIVE Nirbhaya's rapists to be hanged on Jan...,"[massive, nirbhaya, rapists, hanged, jan, 22, ...","[(massive, JJ), (nirbhaya, NN), (rapists, NNS)...","[nirbhaya, rapists, jan, court, issues, death]",1
666,check out pit blocks 3d by srm games! follow @...,Anne's game,1214463810049736704,2020-01-07 08:28:30,"['indiegame', 'indiedev', 'gamedev', 'indievid...",lesptitboutchou,0.0,1.0,NaN,-0.3578,Negative,2,0.000000,0.000138,0.005501,Check out Pit Blocks 3D by SRM Games Follow @i...,"[check, pit, blocks, 3d, srm, games, follow, i...","[(check, NN), (pit, NN), (blocks, VBZ), (3d, C...","[check, pit, srm, games, indievideogames, game...",1
669,strangely tm krishna will shut tight like a cl...,Natarajan நટरा જन,1214448465108860928,2020-01-07 07:27:32,[],NTanjore,0.0,1.0,NaN,0.4753,Positive,4,0.000000,0.000138,0.005501,Strangely TM krishna will shut tight like a cl...,"[strangely, tm, krishna, shut, tight, like, cl...","[(strangely, RB), (tm, JJ), (krishna, NN), (sh...","[krishna, issues, girls, suicides, college, ro...",1
731,"sir, u have every right to condemn everything....",Ramesh Babu,1214216060234133504,2020-01-06 16:04:02,[],actuallycitizen,0.0,2.0,NaN,-0.9775,Negative,1,0.000000,0.000275,0.011001,Sir u have every right to condemn everything B...,"[sir, u, every, right, condemn, everything, pr...","[(sir, NN), (u, JJ), (every, DT), (right, JJ),...","[sir, condemn, everything, problem, show, deba...",1
784,bookings varuku bunny movies lo dj \nah gundu ...,¶®,1214047685851803648,2020-01-06 04:54:58,[],SaiAA_DHF,34.0,81.0,NaN,0.0000,Neutral,3,0.024234,0.011139,1.899572,Bookings varuku Bunny Movies lo DJ Ah gundu ga...,"[bookings, varuku, bunny, movies, lo, dj, ah, ...","[(bookings, NNS), (varuku, VBP), (bunny, JJ), ...","[bookings, movies, ah, gundu, gaadu, kncm, baa...",1
811,check out pit blocks 3d by srm games! follow @...,Picking Pies,1213937325320933376,2020-01-05 21:36:26,"['indiegame', 'indiedev', 'gamedev', 'indievid...",PickingPies,0.0,0.0,NaN,-0.3578,Negative,2,0.000000,0.000000,0.000000,Check out Pit Blocks 3D by SRM Games F

In [19]:
def assignpred(n):
    if n==2:
        return 1
    if n==4:
        return 5
    return n

In [20]:
engagement_median = df['engagement_score'].median()
df["engagement"] = df['engagement_score'].apply(lambda x: 1 if x > engagement_median else 0)

In [21]:
df['new_score']=df['score']

In [22]:
df_eng1=df[df.engagement==1]

In [23]:
df_eng1['new_score']=df_eng1['score'].apply(lambda x: assignpred(x))

/Users/Manam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [24]:
df[df.engagement==1]=df_eng1

In [25]:
#if the tweet is by university, assign sentiment as neutral
df.loc[df['uni_handle']==1,'new_score']=3

In [26]:
#if the tweet contains irrelevant terms, assign sentiment as neutral
df.loc[df['irrelevance']!=0,'new_score']=3

In [27]:
df.rename(columns = {'new_score':'sentiment_score'}, inplace = True) 
df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,...,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens,pos,nouns,irrelevance,engagement,sentiment_score
0,oh snappic.twitter.com/xxmmcpggfj,ZAM the Kitesooner,1228465788773904384,2020-02-14 23:47:22,[],Srm_the_Kitsune,0.0,1.0,NaN,0.0000,...,0.000000,0.000138,0.005501,Oh snappic.twitter.com/xxMMCPGGFJ,"[oh, snappic, twitter, com, xxmmcpggfj]","[(oh, UH), (snappic, NN), (twitter, NN), (com,...","[snappic, twitter, com, xxmmcpggfj]",0,0,3
1,great job santa rosa branch! tgif!!\n.\n.\n.\n...,SMRestorationService,1228443676411662336,2020-02-14 22:19:30,"['werestore', 'servicemasterrestore', 'SRM', '...",SVMRestoration,1.0,0.0,NaN,0.8753,...,0.000713,0.000000,0.042766,Great job Santa Rosa Branch TGIF!! . . . . . ....,"[great, job, santa, rosa, branch, tgif, werest...","[(great, JJ), (job, NN), (santa, NN), (rosa, N...","[job, santa, rosa, branch, tgif, disasterresto...",0,1,5
2,i saw that on msnbc or cnn very early this s m...,June Rzosa,1228437748274663424,2020-02-14 21:55:57,[],RzosaJune,0.0,0.0,NaN,-0.3818,...,0.000000,0.000000,0.000000,I saw that on MSNBC or CNN very early this s m...,"[saw, msnbc, cnn, early, 2, group, kids, stood...","[(saw, NN), (msnbc, NN), (cnn, VBD), (early, J...","[saw, msnbc, group, kids, arm, praise, trump, ...",0,0,2
3,set an example-you first! start re-distributi...,S.R. Mauricio,1228427424171511808,2020-02-14 21:14:55,[],columbus_srm,0.0,0.0,NaN,0.5411,...,0.000000,0.000000,0.000000,Set an example-you first Start re-distributing...,"[set, example, first, start, distributing, wea...","[(set, NN), (example, NN), (first, JJ), (start...","[set, example, start, wealth, billionaire, hyp...",0,0,4
4,another great cover done by yours truly... #re...,Giuseppe D.,1228421427293872128,2020-02-14 20:51:05,"['Repost', 'newmusic', 'newmusicfriday', 'danc...",giuseppedmusic,0.0,1.0,NaN,0.7906,...,0.000000,0.000138,0.005501,Another great cover done by yours truly.. #Rep...,"[another, great, cover, done, truly, repost, r...","[(another, DT), (great, JJ), (cover, NN), (don...","[cover, rayguell, album, edm, musicismylife, n...",0,0,5
5,*double ear infection- confirmed with my 2nd v...,sara,1228403195552182272,2020-02-14 19:38:39,[],ssssara_xo,0.0,0.0,NaN,0.0000,...,0.000000,0.000000,0.000000,*double ear infection confirmed with my 2nd vi...,"[double, ear, infection, confirmed, 2nd, visit...","[(double, JJ), (ear, JJ), (infection, NN), (co...","[infection, visit, clinic, week]",0,0,3
6,available worldwide now and new album coming s...,Ray Guell,1228398941609263104,2020-02-14 19:21:44,"['newmusic', 'newmusicfriday', 'dancemusic', '...",rayguell,0.0,2.0,NaN,0.0000,...,0.000000,0.000275,0.011001,Available worldwide now and new album coming s...,"[available, worldwide, new, album, coming, soo...","[(available, JJ), (worldwide, JJ), (new, JJ), ...","[album, edm, srm, miami, beach, florida]",0,1,3
7,the best pua books on meeting womenhttp://rviv...,Bryce @ SelfReliantMan.com,1228394851055669248,2020-02-14 19:05:29,[],brycefoster_srm,0.0,0.0,NaN,0.6369,...,0.000000,0.000000,0.000000,The Best PUA Books On Meeting,"[best, pua, books, meeting]","[(best, JJS), (pua, NN), (books, NNS), (meetin...","[pua, books, meeting]",0,0,5
8,no more ferrying by private car: gandhi family...,Muhundaram_SeethaRaman,1228384074013630464,2020-02-14 18:22:40,[],SRM_MUHUNDARAM,0.0,0.0,NaN,0.0516,...,0.000000,0.000000,0.000000,No More Ferrying By Private Car Gandhi Family ...,"[ferrying, private, car, gandhi, family, set, ...","[(ferrying, VBG), (private, JJ), (car, NN), (g...","[car, gandhi, family, queue, security, swarajy...",0,0,3
9,great results for our 1st team netballers and ...,St Edmund’s Sport,1228377686495682560,2020-02-14 17:57:17,"['beallyoucanbesport', 'rest', 'recovery']",StEdmundsSport,2.0,5.0,NaN,0.8313,...,0.001426,0.000688,0.113034,Great results for our 1st team netballers and ...,"[great, results, 1st, team, netballers, boys, ...","[(

In [28]:
df.to_csv('../../Datasets/Twitter/Weighted Sentiment/SRMWeightedSentiment.csv')